In [1]:
#Importing dependencies
#Tensorflow
import tensorflow as tf
#importing model builder from the tensorflow serving
from tensorflow.python.saved_model import builder as saved_model_builder
#signature_constants are used to sign models inputs and outputs
from tensorflow.python.saved_model import signature_constants
#nesto
from tensorflow.python.saved_model import signature_def_utils
#give tags to each input/output so we can call it from the client api
from tensorflow.python.saved_model import tag_constants
#tensor builder
from tensorflow.python.saved_model.utils import build_tensor_info

In [2]:
#Import model
from model import ImageSearchModel
#import config
import config as cfg

In [3]:
#Define the model
model = ImageSearchModel(cfg.LEARNING_RATE, cfg.IMAGE_SIZE, cfg.NUMBER_OF_CLASSES)

In [4]:
model.dense_2_features.name

'dense_1/Relu:0'

In [5]:
model.dense_4_features.name

'dense_3/Relu:0'

In [6]:
model.dropout_rate.name

'dropout_probs:0'

In [7]:
#Define session
session = tf.Session()

In [8]:
#init model params
session.run(tf.global_variables_initializer())

In [9]:
#define saver
saver = tf.train.Saver()

In [10]:
#restore the model
saver.restore(session, 'saver/model_epoch_5.ckpt')

INFO:tensorflow:Restoring parameters from saver/model_epoch_5.ckpt


In [11]:
#get model inputs and outputs by tensor name
model_inputs = session.graph.get_tensor_by_name("images:0")
model_inputs_drop_rate = session.graph.get_tensor_by_name('dropout_probs:0')
model_dense_2 = session.graph.get_tensor_by_name('dense_1/Relu:0')
model_dense_4 = session.graph.get_tensor_by_name('dense_3/Relu:0')

In [12]:
#build meta data around the fatched tensors
model_input_ = build_tensor_info(model_inputs)
model_inputs_drop_ = build_tensor_info(model_inputs_drop_rate)
model_output_dense_2 = build_tensor_info(model_dense_2)
model_output_dense_4 = build_tensor_info(model_dense_4)

In [13]:
model_input_

name: "images:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
  dim {
    size: 32
  }
  dim {
    size: 32
  }
  dim {
    size: 3
  }
}

In [14]:
model_inputs_drop_

name: "dropout_probs:0"
dtype: DT_FLOAT
tensor_shape {
  unknown_rank: true
}

In [15]:
model_output_dense_2

name: "dense_1/Relu:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
  dim {
    size: 256
  }
}

In [16]:
model_output_dense_4

name: "dense_3/Relu:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
  dim {
    size: 1024
  }
}

In [17]:
#define model signature
signature_def = signature_def_utils.build_signature_def(
                inputs ={"images:0":model_input_, "drop:0":model_inputs_drop_},
                outputs={"dense1:0":model_output_dense_2, "dense3:0":model_output_dense_4},
                method_name=signature_constants.PREDICT_METHOD_NAME
            )

In [19]:
#build model saver with specific location and model version (2 here)
builder = saved_model_builder.SavedModelBuilder('./models/simple_model/2')

In [20]:
#add meta data to the graph, such as SERVING the model, model signature
builder.add_meta_graph_and_variables(session,
                                     [tag_constants.SERVING], 
                                     signature_def_map={signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:signature_def})

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.


In [21]:
#finally save the model
builder.save()

INFO:tensorflow:SavedModel written to: ./models/simple_model/2/saved_model.pb


b'./models/simple_model/2/saved_model.pb'